In [ ]:
!./build.sh

In [ ]:
!./format.sh
!echo "=== Testing Removing Unused Import and Varialbe ==="
!autoflake --in-place --remove-unused-variables --remove-all-unused-imports save_docx.py

In [ ]:
import os

from deploykit import (
    build_package,
    upload_package, 
    deploy_pacakge, 
    update_package_code,
    update_env_vars
)

# Test Lambda

In [ ]:
from docx import Document as read_document
from docx.document import Document
from io import BytesIO
import base64
import json

In [ ]:
from save_docx import handler

In [ ]:
def python_docx_to_byte(doc: Document) -> bytes:
    byte_stream = BytesIO()
    doc.save(byte_stream)
    byte_stream.seek(0)
    return byte_stream.read()

In [ ]:
def bytes_to_base64_str(byte_data: bytes) -> str:
    b64_encoded_byte = base64.b64encode(byte_data)
    b64_encoded_str = b64_encoded_byte.decode("utf-8")
    return b64_encoded_str

In [ ]:
test_event_dict = {
    "sk": os.environ["TEMP_PROJECT_NAME"],
    "doc": "demo.docx",
    "data": bytes_to_base64_str(
        python_docx_to_byte(
            read_document("../../../lfs/documents/english-demo.docx")
        )
    ),
    "updated_by": "pluto"
}

test_event = {"body": json.dumps(test_event_dict)}

In [ ]:
response = handler(event=test_event, context=_)

In [ ]:
assert response['statusCode'] == 200 

# Deploy Lambda Handler

In [ ]:
# !pip install --upgrade -r requirements.txt --target ./packages

In [ ]:
BUCKET = os.environ['BUCKET']
BUCKET_KEY = "lambdas/save-docx.zip"
FUNCTION_NAME = "quack-tsln-save-document"
ENV_DYNAMODB = os.environ["DB_NAME"]

ENV_VARIABLES = {
    'Variables': {
        'BUCKET': BUCKET, 
        'DB_NAME': ENV_DYNAMODB, 
        "LAMBDA_RUNTIME_ENV": "cloud"
    }
}
CODE_INFO = {"S3Bucket": BUCKET, "S3Key": BUCKET_KEY}
PACKAGE = BUCKET_KEY.replace("lambdas/", "")

In [ ]:
build_package(PACKAGE)

In [ ]:
upload_package(PACKAGE, BUCKET, BUCKET_KEY)

In [ ]:
DESC = "Save Dossier From Client"
MAIN_PYFILE = "save_docx"

In [ ]:
try:
    deploy_pacakge(
        function_name=FUNCTION_NAME, 
        handler=MAIN_PYFILE,
        timeout=180, 
        desc=DESC, 
        env_vars=ENV_VARIABLES,
        code_info=CODE_INFO,
        memory_size=256
    )
except Exception as e:
    print(e)
    update_package_code(function_name=FUNCTION_NAME, bucket=BUCKET, key=BUCKET_KEY)

In [ ]:
update_env_vars(function_name=FUNCTION_NAME, envvars=ENV_VARIABLES)

# Dummy Test for FrontEnd

In [ ]:
import boto3
from save_dossier import create_project_from_query
from save_dossier import Project
import logging

In [ ]:
# Boto3 setting
SESSION = boto3.session.Session()
S3_CLEINT = SESSION.client('s3')
LAMBDA_CLIENT = SESSION.client('lambda')

In [ ]:
resp = create_project_from_query(sk="...")
project = Project.from_dict(resp["data"])

In [ ]:
def download_document(bucket: str, bucket_key: str, local_path: str):
    """Download dictionary from a given bucket and bucket key"""
    S3_CLEINT.download_file(Bucket=bucket, Key=bucket_key, Filename=local_path)
    logging.info(f"File downloaded successfully from bucket {bucket} with key {bucket_key}")

In [ ]:
download_document(project.bucket, project.bucket_key, "temp/dossier.docx")